<center><h1>第五章 变形</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、长宽表的变形

什么是长表？什么是宽表？这个概念是对于某一个特征而言的。例如：一个表中把性别存储在某一个列中，那么它就是关于性别的长表；如果把性别作为列名，列中的元素是某一其他的相关特征数值，那么这个表是关于性别的宽表。下面的两张表就分别是关于性别的长表和宽表：

In [2]:
pd.DataFrame({'Gender':['F','F','M','M'], 'Height':[163, 160, 175, 180]})

Gender  Height
0      F     163
1      F     160
2      M     175
3      M     180

In [3]:
pd.DataFrame({'Height: F':[163, 160], 'Height: M':[175, 180]})

Height: F  Height: M
0        163        175
1        160        180

显然这两张表从信息上是完全等价的，它们包含相同的身高统计数值，只是这些数值的呈现方式不同，而其呈现方式主要又与性别一列选择的布局模式有关，即到底是以$\color{red}{long}$的状态存储还是以$\color{red}{wide}$的状态存储。因此，`pandas`针对此类长宽表的变形操作设计了一些有关的变形函数。

### 1. pivot

`pivot`是一种典型的长表变宽表的函数，首先来看一个例子：下表存储了张三和李四的语文和数学分数，现在想要把语文和数学分数作为列来展示。

In [4]:
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

Class       Name  Subject  Grade
0      1  San Zhang  Chinese     80
1      1  San Zhang     Math     75
2      2      Si Li  Chinese     90
3      2      Si Li     Math     85

对于一个基本的长变宽操作而言，最重要的有三个要素，分别是变形后的行索引、需要转到列索引的列，以及这些列和行索引对应的数值，它们分别对应了`pivot`方法中的`index, columns, values`参数。新生成表的列索引是`columns`对应列的`unique`值，而新表的行索引是`index`对应列的`unique`值，而`values`对应了想要展示的数值列。

In [5]:
df.pivot(index='Name', columns='Subject', values='Grade')

Subject    Chinese  Math
Name                    
San Zhang       80    75
Si Li           90    85

通过颜色的标记，更容易地能够理解其变形的过程：

<img src="../source/_static/ch5_pivot.png" width="50%">

利用`pivot`进行变形操作需要满足唯一性的要求，即由于在新表中的行列索引对应了唯一的`value`，因此原表中的`index`和`columns`对应两个列的行组合必须唯一。例如，现在把原表中第二行张三的数学改为语文就会报错，这是由于`Name`与`Subject`的组合中两次出现`("San Zhang", "Chinese")`，从而最后不能够确定到底变形后应该是填写80分还是75分。

In [6]:
df.loc[1, 'Subject'] = 'Chinese'
try:
    df.pivot(index='Name', columns='Subject', values='Grade')
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

`pandas`从`1.1.0`开始，`pivot`相关的三个参数允许被设置为列表，这也意味着会返回多级索引。这里构造一个相应的例子来说明如何使用：下表中六列分别为班级、姓名、测试类型（期中考试和期末考试）、科目、成绩、排名。

In [7]:
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                              'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                    'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                                 'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})
df

Class       Name Examination  Subject  Grade  rank
0      1  San Zhang         Mid  Chinese     80    10
1      1  San Zhang       Final  Chinese     75    15
2      2      Si Li         Mid  Chinese     85    21
3      2      Si Li       Final  Chinese     65    15
4      1  San Zhang         Mid     Math     90    20
5      1  San Zhang       Final     Math     85     7
6      2      Si Li         Mid     Math     92     6
7      2      Si Li       Final     Math     88     2

现在想要把测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引，并且同时统计成绩和排名：

In [8]:
pivot_multi = df.pivot(index = ['Class', 'Name'],
                       columns = ['Subject','Examination'],
                       values = ['Grade','rank'])
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

根据唯一性原则，新表的行索引等价于对`index`中的多列使用`drop_duplicates`，而列索引的长度为`values`中的元素个数乘以`columns`的唯一组合数量（与`index`类似） 。从下面的示意图中能够比较容易地理解相应的操作：

<img src="../source/_static/ch5_mulpivot.png" width="50%">

### 2. pivot_table

`pivot`的使用依赖于唯一性条件，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值。例如，张三和李四都参加了两次语文考试和数学考试，按照学院规定，最后的成绩是两次考试分数的平均值，此时就无法通过`pivot`函数来完成。

In [9]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

Name  Subject  Grade
0  San Zhang  Chinese     80
1  San Zhang  Chinese     90
2  San Zhang     Math    100
3  San Zhang     Math     90
4      Si Li  Chinese     70
5      Si Li  Chinese     80
6      Si Li     Math     85
7      Si Li     Math     95

`pandas`中提供了`pivot_table`来实现，其中的`aggfunc`参数就是使用的聚合函数。上述场景可以如下写出：

In [10]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject    Chinese  Math
Name                    
San Zhang       85    95
Si Li           75    90

这里传入`aggfunc`包含了上一章中介绍的所有合法聚合字符串，此外还可以传入以序列为输入，标量为输出的聚合函数来实现自定义操作，上述功能可以等价写出：

In [11]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x:x.mean())

Subject    Chinese  Math
Name                    
San Zhang       85    95
Si Li           75    90

此外，`pivot_table`具有边际汇总的功能，可以通过设置`margins=True`来实现，其中边际的聚合方式与`aggfunc`中给出的聚合方法一致。下面就分别统计了语文均分和数学均分、张三均分和李四均分，以及总体所有分数的均分：

【我的理解】边际汇总有点类似于边缘分布函数的意思，对于离散型分布函数，画分布列的表格时候的边缘分布。

In [12]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc='mean',
               margins=True)

Subject    Chinese  Math    All
Name                           
San Zhang       85  95.0  90.00
Si Li           75  90.0  82.50
All             80  92.5  86.25

#### 【练一练】
在上面的边际汇总例子中，行或列的汇总为新表中行元素或者列元素的平均值，而总体的汇总为新表中四个元素的平均值。这种关系一定成立吗？若不成立，请给出一个例子来说明。

In [13]:
# 暂时没有想到合适的例子

#### 【END】
### 3. melt

长宽表只是数据呈现方式的差异，但其包含的信息量是等价的，前面提到了利用`pivot`把长表转为宽表，那么就可以通过相应的逆操作把宽表转为长表，`melt`函数就起到了这样的作用。在下面的例子中，`Subject`以列索引的形式存储，现在想要将其压缩到一个列中。

In [14]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

Class       Name  Chinese  Math
0      1  San Zhang       80    80
1      2      Si Li       90    75

In [16]:
df

Class       Name  Chinese  Math
0      1  San Zhang       80    80
1      2      Si Li       90    75

In [15]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')
df_melted

Class       Name  Subject  Grade
0      1  San Zhang  Chinese     80
1      2      Si Li  Chinese     90
2      1  San Zhang     Math     80
3      2      Si Li     Math     75

`melt`的主要参数和压缩的过程如下图所示：

<img src="../source/_static/ch5_melt.png" width="50%">

前面提到了`melt`和`pivot`是一组互逆过程，那么就一定可以通过`pivot`操作把`df_melted`转回`df`的形式：

In [17]:
df_unmelted = df_melted.pivot(index = ['Class', 'Name'],
                              columns='Subject',
                              values='Grade')
df_unmelted # 下面需要恢复索引，并且重命名列索引名称

Subject          Chinese  Math
Class Name                    
1     San Zhang       80    80
2     Si Li           90    75

In [18]:
df_unmelted = df_unmelted.reset_index().rename_axis(columns={'Subject':''})
df_unmelted.equals(df)

True

### 4. wide_to_long

`melt`方法中，在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，即`values_name`。现在如果列中包含了交叉类别，比如期中期末的类别和语文数学的类别，那么想要把`values_name`对应的`Grade`扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩，这种需求下就要使用`wide_to_long`函数来完成。

https://pandas.pydata.org/docs/reference/api/pandas.wide_to_long.html

In [19]:
df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang', 'Si Li'],
                   'Chinese_Mid':[80, 75], 'Math_Mid':[90, 85],
                   'Chinese_Final':[80, 75], 'Math_Final':[90, 85]})
df

Class       Name  Chinese_Mid  Math_Mid  Chinese_Final  Math_Final
0      1  San Zhang           80        90             80          90
1      2      Si Li           75        85             75          85

In [20]:
pd.wide_to_long(df,
                stubnames=['Chinese', 'Math'],
                i = ['Class', 'Name'],
                j='Examination',
                sep='_',
                suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

具体的变换过程由下图进行展示，属相同概念的元素使用了一致的颜色标出：

<img src="../source/_static/ch5_wtl.png" width="50%">

下面给出一个比较复杂的案例，把之前在`pivot`一节中多列操作的结果（产生了多级索引），利用`wide_to_long`函数，将其转为原来的形态。其中，使用了第八章的`str.split`函数，目前暂时只需将其理解为对序列按照某个分隔符进行拆分即可。

In [23]:
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

In [21]:
res = pivot_multi.copy()
res.columns = res.columns.map(lambda x:'_'.join(x))
res = res.reset_index()
res

Class       Name  Grade_Chinese_Mid  Grade_Chinese_Final  Grade_Math_Mid  \
0      1  San Zhang                 80                   75              90   
1      2      Si Li                 85                   65              92   

   Grade_Math_Final  rank_Chinese_Mid  rank_Chinese_Final  rank_Math_Mid  \
0                85                10                  15             20   
1                88                21                  15              6   

   rank_Math_Final  
0                7  
1                2

In [22]:

res = pd.wide_to_long(res, stubnames=['Grade', 'rank'],
                           i = ['Class', 'Name'],
                           j = 'Subject_Examination',
                           sep = '_',
                           suffix = '.+')
res

Grade  rank
Class Name      Subject_Examination             
1     San Zhang Chinese_Mid             80    10
                Chinese_Final           75    15
                Math_Mid                90    20
                Math_Final              85     7
2     Si Li     Chinese_Mid             85    21
                Chinese_Final           65    15
                Math_Mid                92     6
                Math_Final              88     2

In [24]:
res = res.reset_index()
res

Class       Name Subject_Examination  Grade  rank
0      1  San Zhang         Chinese_Mid     80    10
1      1  San Zhang       Chinese_Final     75    15
2      1  San Zhang            Math_Mid     90    20
3      1  San Zhang          Math_Final     85     7
4      2      Si Li         Chinese_Mid     85    21
5      2      Si Li       Chinese_Final     65    15
6      2      Si Li            Math_Mid     92     6
7      2      Si Li          Math_Final     88     2

In [25]:
res[['Subject', 'Examination']] = res['Subject_Examination'].str.split('_', expand=True)
res = res[['Class', 'Name', 'Examination', 'Subject', 'Grade', 'rank']].sort_values('Subject')
res = res.reset_index(drop=True)
res

Class       Name Examination  Subject  Grade  rank
0      1  San Zhang         Mid  Chinese     80    10
1      1  San Zhang       Final  Chinese     75    15
2      2      Si Li         Mid  Chinese     85    21
3      2      Si Li       Final  Chinese     65    15
4      1  San Zhang         Mid     Math     90    20
5      1  San Zhang       Final     Math     85     7
6      2      Si Li         Mid     Math     92     6
7      2      Si Li       Final     Math     88     2

`reset_index` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

## 二、索引的变形

### 1. stack与unstack

在第二章中提到了利用`swaplevel`或者`reorder_levels`进行索引内部的层交换，下面就要讨论$\color{red}{行列索引之间}$的交换，由于这种交换带来了`DataFrame`维度上的变化，因此属于变形操作。在第一节中提到的4种变形函数与其不同之处在于，它们都属于某一列或几列$\color{red}{元素}$和$\color{red}{列索引}$之间的转换，而不是索引之间的转换。

`unstack`函数的作用是把行索引转为列索引，例如下面这个简单的例子：

In [26]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['col_1', 'col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [27]:
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

`unstack`的主要参数是移动的层号，默认转化最内层，移动到列索引的最内层，同时支持同时转化多个层：

In [29]:
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [30]:
df.unstack(2)

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [31]:
df.unstack(1)

col_1      col_2     
          cat  dog   cat  dog
A big     1.0  NaN   1.0  NaN
  small   NaN  1.0   NaN  1.0
B big     1.0  NaN   1.0  NaN
  small   NaN  1.0   NaN  1.0

In [32]:
df.unstack(0)

col_1      col_2     
              A    B     A    B
cat big     1.0  1.0   1.0  1.0
dog small   1.0  1.0   1.0  1.0

In [24]:
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

`unstack` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html

关于 参数 level 的编号，0 表示最外层，越大的数字表示越靠内的层。

类似于`pivot`中的唯一性要求，在`unstack`中必须保证$\color{red}{被转为列索引的行索引层}$和$\color{red}{被保留的行索引层}$构成的组合是唯一的，例如把前两个列索引改成相同的破坏唯一性，那么就会报错：

In [36]:
df.index.to_list()[1]

('A', 'dog', 'small')

In [37]:
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [38]:
my_index = df.index.to_list()
my_index[1] = my_index[0]
df.index = pd.Index(my_index)
df

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [39]:
try:
    df.unstack()
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

与`unstack`相反，`stack`的作用就是把列索引的层压入行索引，其用法完全类似。

In [40]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['index_1', 'index_2']).T
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [41]:
df.stack()

A         B     
               cat  dog  cat  dog
index_1 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0
index_2 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0

In [42]:
df.stack([1, 2])

A    B
index_1 cat big    1.0  1.0
        dog small  1.0  1.0
index_2 cat big    1.0  1.0
        dog small  1.0  1.0

In [43]:
df.stack(0)

cat   dog
           big small
index_1 A  1.0   1.0
        B  1.0   1.0
index_2 A  1.0   1.0
        B  1.0   1.0

`stack` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html

参数 `level` 也是相同的用法。从外到内编号从 0 开始，默认为 -1，表示最内层。

### 2. 聚合与变形的关系

在上面介绍的所有函数中，除了带有聚合效果的`pivot_table`以外，所有的函数在变形前后并不会带来`values`个数的改变，只是这些值在呈现的形式上发生了变化。在上一章讨论的分组聚合操作，由于生成了新的行列索引，因此必然也属于某种特殊的变形操作，但由于聚合之后把原来的多个值变为了一个值，因此`values`的个数产生了变化，这也是分组聚合与变形函数的最大区别。

## 三、其他变形函数

### 1. crosstab

`crosstab`是一个地位尴尬的函数，因为它能实现的所有功能`pivot_table`都能完成。在默认状态下，`crosstab`可以统计元素组合出现的频数，即`count`操作。例如统计`learn_pandas`数据集中学校和转系情况对应的频数：

In [44]:
df = pd.read_csv('../data/learn_pandas.csv')
df

School      Grade            Name  Gender  Height  \
0    Shanghai Jiao Tong University   Freshman    Gaopeng Yang  Female   158.9   
1                Peking University   Freshman  Changqiang You    Male   166.5   
2    Shanghai Jiao Tong University     Senior         Mei Sun    Male   188.9   
3                 Fudan University  Sophomore    Xiaojuan Sun  Female     NaN   
4                 Fudan University  Sophomore     Gaojuan You    Male   174.0   
..                             ...        ...             ...     ...     ...   
195               Fudan University     Junior    Xiaojuan Sun  Female   153.9   
196            Tsinghua University     Senior         Li Zhao  Female   160.9   
197  Shanghai Jiao Tong University     Senior  Chengqiang Chu  Female   153.9   
198  Shanghai Jiao Tong University     Senior   Chengmei Shen    Male   175.3   
199            Tsinghua University  Sophomore     Chunpeng Lv    Male   155.7   

     Weight Transfer  Test_Number   Test_Date Time_Record  
0      46.0        N            1   2019/10/5     0:04:34  
1      70.0        N            1    2019/9/4     0:04:20  
2      89.0        N            2   2019/9/12     0:05:22  
3      41.0        N            2    2020/1/3     0:04:08  
4      74.0        N            2   2019/11/6     0:05:22  
..      ...      ...          ...         ...         ...  
195    46.0        N            2  2019/10/17     0:04:31  
196    50.0        N            3   2019/9/22     0:04:03  
197    45.0        N            1    2020/1/5     0:04:48  
198    71.0        N            2    2020/1/7     0:04:58  
199    51.0        N            1   2019/11/6     0:05:05  

[200 rows x 10 columns]

In [45]:
pd.crosstab(index = df.School, columns = df.Transfer)

Transfer                        N  Y
School                              
Fudan University               38  1
Peking University              28  2
Shanghai Jiao Tong University  53  0
Tsinghua University            62  4

In [46]:
pd.crosstab(index = df.School, columns = df.Gender)

Gender                         Female  Male
School                                     
Fudan University                   30    10
Peking University                  22    12
Shanghai Jiao Tong University      41    16
Tsinghua University                48    21

上面的是统计男女人数。

这等价于如下`crosstab`的如下写法，这里的`aggfunc`即聚合参数：

In [47]:
pd.crosstab(index = df.School, columns = df.Transfer, values = [0]*df.shape[0], aggfunc = 'count')

Transfer                          N    Y
School                                  
Fudan University               38.0  1.0
Peking University              28.0  2.0
Shanghai Jiao Tong University  53.0  NaN
Tsinghua University            62.0  4.0

同样，可以利用`pivot_table`进行等价操作，由于这里统计的是组合的频数，因此`values`参数无论传入哪一个列都不会影响最后的结果：

In [48]:
df.pivot_table(index = 'School',
               columns = 'Transfer',
               values = 'Name',
               aggfunc = 'count')

Transfer                          N    Y
School                                  
Fudan University               38.0  1.0
Peking University              28.0  2.0
Shanghai Jiao Tong University  53.0  NaN
Tsinghua University            62.0  4.0

In [50]:
df.pivot_table(index = 'School',
               columns = 'Transfer',
               values = 'Test_Number', # 这个values可以是别的，出来结果是一样的，但是从数据逻辑上，名字是更合理的
               aggfunc = 'count')

Transfer                          N    Y
School                                  
Fudan University               38.0  1.0
Peking University              28.0  2.0
Shanghai Jiao Tong University  53.0  NaN
Tsinghua University            62.0  4.0

从上面可以看出这两个函数的区别在于，`crosstab`的对应位置传入的是具体的序列，而`pivot_table`传入的是被调用表对应的名字，若传入序列对应的值则会报错。

除了默认状态下的`count`统计，所有的聚合字符串和返回标量的自定义函数都是可用的，例如统计对应组合的身高均值：

In [51]:
pd.crosstab(index = df.School, columns = df.Transfer, values = df.Height, aggfunc = 'mean')

Transfer                                N       Y
School                                           
Fudan University               162.043750  177.20
Peking University              163.429630  162.40
Shanghai Jiao Tong University  163.953846     NaN
Tsinghua University            163.253571  164.55

In [52]:
pd.crosstab(index = df.School, columns = df.Gender, values = df.Height, aggfunc = 'mean')

Gender                             Female        Male
School                                               
Fudan University               158.776923  174.212500
Peking University              158.666667  172.030000
Shanghai Jiao Tong University  159.122500  176.760000
Tsinghua University            159.753333  171.638889

### 2. explode

`explode`参数能够对某一列的元素进行纵向的展开，被展开的单元格必须存储`list, tuple, Series, np.ndarray`中的一种类型。

In [53]:
df_ex = pd.DataFrame({'A': [[1, 2], 'my_str', {1, 2}, pd.Series([3, 4])],
                      'B': 1})
df_ex

A  B
0                      [1, 2]  1
1                      my_str  1
2                      {1, 2}  1
3  0    3
1    4
dtype: int64  1

In [54]:
df_ex.explode('A')

A  B
0       1  1
0       2  1
1  my_str  1
2       1  1
2       2  1
3       3  1
3       4  1

### 3. get_dummies

`get_dummies`是用于特征构建的重要函数之一，其作用是把类别特征转为指示变量。例如，对年级一列转为指示变量，属于某一个年级的对应列标记为1，否则为0：

In [55]:
df

School      Grade            Name  Gender  Height  \
0    Shanghai Jiao Tong University   Freshman    Gaopeng Yang  Female   158.9   
1                Peking University   Freshman  Changqiang You    Male   166.5   
2    Shanghai Jiao Tong University     Senior         Mei Sun    Male   188.9   
3                 Fudan University  Sophomore    Xiaojuan Sun  Female     NaN   
4                 Fudan University  Sophomore     Gaojuan You    Male   174.0   
..                             ...        ...             ...     ...     ...   
195               Fudan University     Junior    Xiaojuan Sun  Female   153.9   
196            Tsinghua University     Senior         Li Zhao  Female   160.9   
197  Shanghai Jiao Tong University     Senior  Chengqiang Chu  Female   153.9   
198  Shanghai Jiao Tong University     Senior   Chengmei Shen    Male   175.3   
199            Tsinghua University  Sophomore     Chunpeng Lv    Male   155.7   

     Weight Transfer  Test_Number   Test_Date Time_Record  
0      46.0        N            1   2019/10/5     0:04:34  
1      70.0        N            1    2019/9/4     0:04:20  
2      89.0        N            2   2019/9/12     0:05:22  
3      41.0        N            2    2020/1/3     0:04:08  
4      74.0        N            2   2019/11/6     0:05:22  
..      ...      ...          ...         ...         ...  
195    46.0        N            2  2019/10/17     0:04:31  
196    50.0        N            3   2019/9/22     0:04:03  
197    45.0        N            1    2020/1/5     0:04:48  
198    71.0        N            2    2020/1/7     0:04:58  
199    51.0        N            1   2019/11/6     0:05:05  

[200 rows x 10 columns]

In [56]:
pd.get_dummies(df.Grade).head()

Freshman  Junior  Senior  Sophomore
0         1       0       0          0
1         1       0       0          0
2         0       0       1          0
3         0       0       0          1
4         0       0       0          1

In [57]:
pd.get_dummies(df.Grade)

Freshman  Junior  Senior  Sophomore
0           1       0       0          0
1           1       0       0          0
2           0       0       1          0
3           0       0       0          1
4           0       0       0          1
..        ...     ...     ...        ...
195         0       1       0          0
196         0       0       1          0
197         0       0       1          0
198         0       0       1          0
199         0       0       0          1

[200 rows x 4 columns]

## 四、练习
### Ex1：美国非法药物数据集

现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量：

In [103]:
df = pd.read_csv('../data/drugs.csv').sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
df.head(3)

YYYY State COUNTY  SubstanceName  DrugReports
0  2011    KY  ADAIR  Buprenorphine            3
1  2012    KY  ADAIR  Buprenorphine            5
2  2013    KY  ADAIR  Buprenorphine            4

In [104]:
df

YYYY State   COUNTY  SubstanceName  DrugReports
0      2011    KY    ADAIR  Buprenorphine            3
1      2012    KY    ADAIR  Buprenorphine            5
2      2013    KY    ADAIR  Buprenorphine            4
3      2014    KY    ADAIR  Buprenorphine           27
4      2015    KY    ADAIR  Buprenorphine            5
...     ...   ...      ...            ...          ...
24057  2010    WV  WYOMING      Oxycodone            5
24058  2011    WV  WYOMING      Oxycodone            4
24059  2012    WV  WYOMING      Oxycodone           14
24060  2013    WV  WYOMING      Oxycodone           12
24061  2014    WV  WYOMING      Oxycodone            5

[24062 rows x 5 columns]

1. 将数据转为如下的形式：

<img src="../source/_static/Ex5_1.png" width="50%">

In [112]:
# 相当于是把长表变为宽表的操作 把 年份这一列的值抽出来作为行索引，然后报告数量作为表的值。但是从结果看又不完全是这样

df_1 = df.pivot(index=['State', 'COUNTY', 'SubstanceName'], columns='YYYY', values='DrugReports')

In [113]:
df_1

YYYY                         2010  2011  2012  2013  2014  2015  2016  2017
State COUNTY  SubstanceName                                                
KY    ADAIR   Buprenorphine   NaN   3.0   5.0   4.0  27.0   5.0   7.0  10.0
              Codeine         NaN   NaN   1.0   NaN   NaN   NaN   NaN   1.0
              Fentanyl        NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN
              Heroin          NaN   NaN   1.0   2.0   NaN   1.0   NaN   2.0
              Hydrocodone     6.0   9.0  10.0  10.0   9.0   7.0  11.0   3.0
...                           ...   ...   ...   ...   ...   ...   ...   ...
WV    WOOD    Oxycodone       6.0   4.0  24.0   7.0   7.0  11.0   7.0   1.0
              Tramadol        NaN   NaN   NaN   NaN   1.0   NaN   4.0   3.0
      WYOMING Buprenorphine   NaN   1.0   1.0   1.0   NaN   NaN   NaN   1.0
              Hydrocodone     1.0   5.0   NaN   NaN   1.0   NaN   1.0   NaN
              Oxycodone       5.0   4.0  14.0  12.0   5.0   NaN   NaN   NaN

[6214 rows x 8 columns]

In [115]:
df_1 = df_1.reset_index()
df_1

YYYY  index State   COUNTY  SubstanceName  2010  2011  2012  2013  2014  2015  \
0         0    KY    ADAIR  Buprenorphine   NaN   3.0   5.0   4.0  27.0   5.0   
1         1    KY    ADAIR        Codeine   NaN   NaN   1.0   NaN   NaN   NaN   
2         2    KY    ADAIR       Fentanyl   NaN   NaN   1.0   NaN   NaN   NaN   
3         3    KY    ADAIR         Heroin   NaN   NaN   1.0   2.0   NaN   1.0   
4         4    KY    ADAIR    Hydrocodone   6.0   9.0  10.0  10.0   9.0   7.0   
...     ...   ...      ...            ...   ...   ...   ...   ...   ...   ...   
6209   6209    WV     WOOD      Oxycodone   6.0   4.0  24.0   7.0   7.0  11.0   
6210   6210    WV     WOOD       Tramadol   NaN   NaN   NaN   NaN   1.0   NaN   
6211   6211    WV  WYOMING  Buprenorphine   NaN   1.0   1.0   1.0   NaN   NaN   
6212   6212    WV  WYOMING    Hydrocodone   1.0   5.0   NaN   NaN   1.0   NaN   
6213   6213    WV  WYOMING      Oxycodone   5.0   4.0  14.0  12.0   5.0   NaN   

YYYY  2016  2017  
0      7.0  10.0  
1      NaN   1.0  
2      NaN   NaN  
3      NaN   2.0  
4     11.0   3.0  
...    ...   ...  
6209   7.0   1.0  
6210   4.0   3.0  
6211   NaN   1.0  
6212   1.0   NaN  
6213   NaN   NaN  

[6214 rows x 12 columns]

2. 将第1问中的结果恢复为原表。

宽表变长表有两种，

`melt` 和 `wide_to_long` 都可以试试。

In [108]:
df_1.shape

(6214, 8)

In [119]:
df.head(5)

YYYY State COUNTY  SubstanceName  DrugReports
0  2011    KY  ADAIR  Buprenorphine            3
1  2012    KY  ADAIR  Buprenorphine            5
2  2013    KY  ADAIR  Buprenorphine            4
3  2014    KY  ADAIR  Buprenorphine           27
4  2015    KY  ADAIR  Buprenorphine            5

In [173]:
df_2 = df_1.melt(id_vars=['State', 'COUNTY', 'SubstanceName'],
          value_vars=[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
          var_name='YYYY',
          value_name='DrugReports')

In [174]:
df_2 = df_2.dropna(axis=0, how='any')

In [175]:
df_2 = df_2.sort_values(by=['State', 'COUNTY', 'SubstanceName'])

In [176]:
df_2.reset_index(drop=True, inplace=True)

In [177]:
df_2 = df_2.reindex_like(df)

In [196]:
df_2['DrugReports'] = df_2['DrugReports'].astype('int64')  # 如果类型不转换，就不相等
df_2['YYYY'] = df_2['YYYY'].astype('int64')  # 如果类型不转换，就不相等

In [197]:
df_2.equals(df)

True

In [202]:
df_1

YYYY  index State   COUNTY  SubstanceName  2010  2011  2012  2013  2014  2015  \
0         0    KY    ADAIR  Buprenorphine   NaN   3.0   5.0   4.0  27.0   5.0   
1         1    KY    ADAIR        Codeine   NaN   NaN   1.0   NaN   NaN   NaN   
2         2    KY    ADAIR       Fentanyl   NaN   NaN   1.0   NaN   NaN   NaN   
3         3    KY    ADAIR         Heroin   NaN   NaN   1.0   2.0   NaN   1.0   
4         4    KY    ADAIR    Hydrocodone   6.0   9.0  10.0  10.0   9.0   7.0   
...     ...   ...      ...            ...   ...   ...   ...   ...   ...   ...   
6209   6209    WV     WOOD      Oxycodone   6.0   4.0  24.0   7.0   7.0  11.0   
6210   6210    WV     WOOD       Tramadol   NaN   NaN   NaN   NaN   1.0   NaN   
6211   6211    WV  WYOMING  Buprenorphine   NaN   1.0   1.0   1.0   NaN   NaN   
6212   6212    WV  WYOMING    Hydrocodone   1.0   5.0   NaN   NaN   1.0   NaN   
6213   6213    WV  WYOMING      Oxycodone   5.0   4.0  14.0  12.0   5.0   NaN   

YYYY  2016  2017  
0      7.0  10.0  
1      NaN   1.0  
2      NaN   NaN  
3      NaN   2.0  
4     11.0   3.0  
...    ...   ...  
6209   7.0   1.0  
6210   4.0   3.0  
6211   NaN   1.0  
6212   1.0   NaN  
6213   NaN   NaN  

[6214 rows x 12 columns]

In [201]:
# 第二种方法 wide to long  
# 不能用 这个方法用于 压缩

#pd.wide_to_long(df_2)

TypeError: wide_to_long() missing 3 required positional arguments: 'stubnames', 'i', and 'j'

3. 按`State`分别统计每年的报告数量总和，其中`State, YYYY`分别为列索引和行索引，要求分别使用`pivot_table`函数与`groupby+unstack`两种不同的策略实现，并体会它们之间的联系。

In [204]:
# pivot_table 方法
df_31 = df.pivot_table(index='YYYY', columns='State', values='DrugReports', aggfunc='sum')
df_31

State     KY     OH     PA     VA    WV
YYYY                                   
2010   10453  19707  19814   8685  2890
2011   10289  20330  19987   6749  3271
2012   10722  23145  19959   7831  3376
2013   11148  26846  20409  11675  4046
2014   11081  30860  24904   9037  3280
2015    9865  37127  25651   8810  2571
2016    9093  42470  26164  10195  2548
2017    9394  46104  27894  10448  1614

In [ ]:
pd.d

In [209]:
# groupby and unstack
df_32 = pd.DataFrame(df.groupby(['State', 'YYYY'])['DrugReports'].sum())

In [223]:
df_32 = df_32.unstack(0).rename(columns={'DrugReports': ''}, level=0)

In [224]:
df_32

State     KY     OH     PA     VA    WV
YYYY                                   
2010   10453  19707  19814   8685  2890
2011   10289  20330  19987   6749  3271
2012   10722  23145  19959   7831  3376
2013   11148  26846  20409  11675  4046
2014   11081  30860  24904   9037  3280
2015    9865  37127  25651   8810  2571
2016    9093  42470  26164  10195  2548
2017    9394  46104  27894  10448  1614

### Ex2：特殊的wide_to_long方法

从功能上看，`melt`方法应当属于`wide_to_long`的一种特殊情况，即`stubnames`只有一类。请使用`wide_to_long`生成`melt`一节中的`df_melted`。（提示：对列名增加适当的前缀）

In [225]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

Class       Name  Chinese  Math
0      1  San Zhang       80    80
1      2      Si Li       90    75

In [242]:
col_nam = df.columns.values.tolist()

In [243]:
col_nam

['Class', 'Name', 'Chinese', 'Math']

In [244]:
col_nam = ['Class', 'Name', 'exam-Chinese', 'exam-Math']

In [245]:
pd.wide_to_long(df,
                stubnames='Exam',
                i=['Class', 'Name'],
                j='Examination',
                sep='-',
                suffix='.+')

Empty DataFrame
Columns: [Math, Chinese, Exam]
Index: []